# Разработка алгоритма термодинамических расчетов

Ноутбук для разработки и тестирования алгоритмов расчета термодинамических реакций.

## Цель
Загрузить сырые данные веществ из двух сессий расчетов и проанализировать их для разработки алгоритма.

## 1. Импорт библиотек

In [11]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import sqlite3

# Добавляем корень проекта в путь
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.thermo_agents.search.database_connector import DatabaseConnector
from src.thermo_agents.search.sql_builder import SQLBuilder
from src.thermo_agents.search.compound_searcher import CompoundSearcher

print("✓ Библиотеки импортированы")

✓ Библиотеки импортированы


## 2. Подключение к базе данных

In [12]:
# Путь к базе данных
db_path = project_root / "data" / "thermo_data.db"

if not db_path.exists():
    raise FileNotFoundError(f"База данных не найдена: {db_path}")

# Инициализация компонентов
db_connector = DatabaseConnector(db_path)
sql_builder = SQLBuilder()
searcher = CompoundSearcher(sql_builder, db_connector)

print(f"✓ База данных подключена: {db_path}")
print(f"✓ Размер БД: {db_path.stat().st_size / (1024*1024):.2f} МБ")

✓ База данных подключена: c:\IDE\repository\agents_for_david\data\thermo_data.db
✓ Размер БД: 57.08 МБ


## 3. Парсинг логов и извлечение данных LLM

### Лог 1: Сессия MgO + SO2
Файл: `logs/sessions/session_20251031_213630_5a2a61.log`

### Лог 2: Сессия Al2O3 + CaF2
Файл: `logs/sessions/session_20251031_213828_e05bf5.log`

In [13]:
import json
import re

def extract_llm_response_from_log(log_file_path: Path) -> dict:
    """
    Извлечь Raw response (JSON) из лог-файла сессии.
    
    Args:
        log_file_path: Путь к файлу лога
        
    Returns:
        Словарь с данными LLM ответа
    """
    with open(log_file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Ищем блок "Raw response (JSON):" до "Extracted parameters:"
    pattern = r'Raw response \(JSON\):\s*(\{.*?\n\})\s*\nExtracted parameters:'
    match = re.search(pattern, content, re.DOTALL)
    
    if not match:
        raise ValueError(f"Не найден Raw response (JSON) в файле {log_file_path}")
    
    # Получаем JSON строку
    json_str = match.group(1)
    
    try:
        data = json.loads(json_str)
        return data
    except json.JSONDecodeError as e:
        print(f"⚠ Ошибка парсинга JSON: {e}")
        print(f"JSON строка (первые 500 символов):\n{json_str[:500]}")
        raise

In [14]:
# Пути к логам
log1_path = project_root / "logs" / "sessions" / "session_20251031_213630_5a2a61.log"
log2_path = project_root / "logs" / "sessions" / "session_20251031_213828_e05bf5.log"

# Извлекаем данные
llm_data_log1 = extract_llm_response_from_log(log1_path)
llm_data_log2 = extract_llm_response_from_log(log2_path)

print("✓ Данные из логов извлечены")
print(f"\nЛог 1 - Вещества: {llm_data_log1['all_compounds']}")
print(f"Лог 2 - Вещества: {llm_data_log2['all_compounds']}")

✓ Данные из логов извлечены

Лог 1 - Вещества: ['MgO', 'SO2', 'MgSO3']
Лог 2 - Вещества: ['Al2O3', 'CaF2', 'AlF3', 'CaO']


### 3.1. Просмотр извлеченных данных из JSON

In [15]:
print("=" * 70)
print("ЛОГ 1 - LLM Response Data")
print("=" * 70)
print(f"Уравнение: {llm_data_log1.get('balanced_equation', 'N/A')}")
print(f"Все вещества: {llm_data_log1.get('all_compounds', [])}")
print(f"\nИмена соединений (compound_names):")
for formula, names in llm_data_log1.get('compound_names', {}).items():
    print(f"  {formula}: {names}")

print("\n" + "=" * 70)
print("ЛОГ 2 - LLM Response Data")
print("=" * 70)
print(f"Уравнение: {llm_data_log2.get('balanced_equation', 'N/A')}")
print(f"Все вещества: {llm_data_log2.get('all_compounds', [])}")
print(f"\nИмена соединений (compound_names):")
for formula, names in llm_data_log2.get('compound_names', {}).items():
    print(f"  {formula}: {names}")

ЛОГ 1 - LLM Response Data
Уравнение: MgO + SO2 → MgSO3
Все вещества: ['MgO', 'SO2', 'MgSO3']

Имена соединений (compound_names):
  MgO: ['Magnesium oxide', 'Magnesia']
  SO2: ['Sulfur dioxide']
  MgSO3: ['Magnesium sulfite']

ЛОГ 2 - LLM Response Data
Уравнение: Al2O3 + 3CaF2 → 2AlF3 + 3CaO
Все вещества: ['Al2O3', 'CaF2', 'AlF3', 'CaO']

Имена соединений (compound_names):
  Al2O3: ['Aluminium oxide', 'Alumina', 'Corundum']
  CaF2: ['Calcium fluoride', 'Fluorite']
  AlF3: ['Aluminium fluoride']
  CaO: ['Calcium oxide', 'Quicklime']


In [16]:
def get_raw_compound_data(formula: str, compound_names: list = None) -> pd.DataFrame:
    """
    Получить все сырые записи соединения из базы данных без фильтрации.
    
    Двухстадийный поиск:
    1. Поиск по формуле + первое имя из compound_names (строгое соответствие)
    2. Если не найдено - поиск только по формуле (fallback)
    
    Args:
        formula: Химическая формула соединения
        compound_names: Список имен соединения из LLM response (опционально)
        
    Returns:
        DataFrame со всеми найденными записей
    """
    db_connector.connect()
    
    # СТАДИЯ 1: Поиск по формуле + первое имя (строгое соответствие)
    if compound_names and len(compound_names) > 0:
        first_name = compound_names[0]
        query_stage1 = f"""
        SELECT * FROM compounds
        WHERE (
            (TRIM(Formula) = '{formula}' OR Formula LIKE '{formula}(%')
            AND (TRIM(FirstName) = '{first_name}' OR TRIM(SecondName) = '{first_name}')
        )
        AND (Formula NOT LIKE '%+%' AND Formula NOT LIKE '%-%')
        ORDER BY 
            CASE ReliabilityClass 
                WHEN 1 THEN 0 WHEN 2 THEN 1 WHEN 3 THEN 2 
                WHEN 0 THEN 3 WHEN 4 THEN 4 WHEN 5 THEN 5 ELSE 6 
            END,
            (Tmax - Tmin) DESC,
            LENGTH(TRIM(Formula)) ASC,
            CASE Phase 
                WHEN 'g' THEN 0 WHEN 'l' THEN 1 
                WHEN 's' THEN 2 WHEN 'aq' THEN 3 ELSE 4 
            END,
            rowid ASC
        """
        
        results = db_connector.execute_query(query_stage1)
        
        if results:
            df = pd.DataFrame(results)
            print(f"✓ {formula} (стадия 1: формула + '{first_name}'): найдено {len(df)} записей")
            return df
        else:
            print(f"  {formula}: стадия 1 (формула + имя) не дала результатов, переход к стадии 2...")
    
    # СТАДИЯ 2: Fallback - поиск только по формуле
    query_stage2 = f"""
    SELECT * FROM compounds
    WHERE (TRIM(Formula) = '{formula}' OR Formula LIKE '{formula}(%')
    AND (Formula NOT LIKE '%+%' AND Formula NOT LIKE '%-%')
    ORDER BY 
        CASE ReliabilityClass 
            WHEN 1 THEN 0 WHEN 2 THEN 1 WHEN 3 THEN 2 
            WHEN 0 THEN 3 WHEN 4 THEN 4 WHEN 5 THEN 5 ELSE 6 
        END,
        (Tmax - Tmin) DESC,
        LENGTH(TRIM(Formula)) ASC,
        CASE Phase 
            WHEN 'g' THEN 0 WHEN 'l' THEN 1 
            WHEN 's' THEN 2 WHEN 'aq' THEN 3 ELSE 4 
        END,
        rowid ASC
    """
    
    results = db_connector.execute_query(query_stage2)
    
    if not results:
        print(f"⚠ {formula}: не найдено записей (обе стадии)")
        return pd.DataFrame()
    
    df = pd.DataFrame(results)
    print(f"✓ {formula} (стадия 2: только формула): найдено {len(df)} записей")
    
    return df

### 3.2. Загрузка данных для Лога 1 (MgO + SO2)

Используем данные из JSON ответа LLM для улучшенного поиска.

In [17]:
# Создаем словарь с данными
log_1 = {}

print("Загрузка данных для Лога 1:")
print("-" * 70)

# Получаем compound_names из LLM ответа
compound_names_dict = llm_data_log1.get('compound_names', {})

for formula in llm_data_log1['all_compounds']:
    names = compound_names_dict.get(formula, None)
    log_1[formula] = get_raw_compound_data(formula, names)

print("\n" + "=" * 70)
print("Сводка по Логу 1:")
for formula, df in log_1.items():
    phases = df['Phase'].unique() if len(df) > 0 else []
    print(f"  {formula}: {len(df)} записей, фазы: {list(phases)}")

Загрузка данных для Лога 1:
----------------------------------------------------------------------
✓ MgO (стадия 1: формула + 'Magnesium oxide'): найдено 71 записей
✓ SO2 (стадия 1: формула + 'Sulfur dioxide'): найдено 1010 записей
✓ MgSO3 (стадия 1: формула + 'Magnesium sulfite'): найдено 12 записей

Сводка по Логу 1:
  MgO: 71 записей, фазы: ['g', 's', 'l']
  SO2: 1010 записей, фазы: ['g', 'l', 'a', 'ao']
  MgSO3: 12 записей, фазы: ['a', 'ai', 's']


### 3.3. Загрузка данных для Лога 2 (Al2O3 + CaF2)

Используем данные из JSON ответа LLM для улучшенного поиска.

In [18]:
# Создаем словарь с данными
log_2 = {}

print("Загрузка данных для Лога 2:")
print("-" * 70)

# Получаем compound_names из LLM ответа
compound_names_dict = llm_data_log2.get('compound_names', {})

for formula in llm_data_log2['all_compounds']:
    names = compound_names_dict.get(formula, None)
    log_2[formula] = get_raw_compound_data(formula, names)

print("\n" + "=" * 70)
print("Сводка по Логу 2:")
for formula, df in log_2.items():
    phases = df['Phase'].unique() if len(df) > 0 else []
    print(f"  {formula}: {len(df)} записей, фазы: {list(phases)}")

Загрузка данных для Лога 2:
----------------------------------------------------------------------
  Al2O3: стадия 1 (формула + имя) не дала результатов, переход к стадии 2...
✓ Al2O3 (стадия 2: только формула): найдено 100 записей
✓ CaF2 (стадия 1: формула + 'Calcium fluoride'): найдено 79 записей
✓ AlF3 (стадия 1: формула + 'Aluminium fluoride'): найдено 30 записей
✓ CaO (стадия 1: формула + 'Calcium oxide'): найдено 85 записей

Сводка по Логу 2:
  Al2O3: 100 записей, фазы: ['g', 'l', 's']
  CaF2: 79 записей, фазы: ['g', 'l', 's', 'ai']
  AlF3: 30 записей, фазы: ['s', 'l']
  CaO: 85 записей, фазы: ['g', 'l', 's']


## 4. Итоговая сводка

Данные загружены с использованием улучшенного двухстадийного алгоритма поиска.

In [19]:
print("=" * 70)
print("ИТОГОВАЯ СВОДКА ПО ЗАГРУЖЕННЫМ ДАННЫМ")
print("=" * 70)

print("\n📊 ЛОГ 1 (Реакция: MgO + SO2 → MgSO3):")
print(f"   Уравнение из LLM: {llm_data_log1.get('balanced_equation')}")
total_log1 = sum(len(df) for df in log_1.values())
print(f"   Всего записей: {total_log1}")
for formula, df in log_1.items():
    names = llm_data_log1.get('compound_names', {}).get(formula, [])
    first_name = names[0] if names else "N/A"
    print(f"   - {formula} ({first_name}): {len(df)} записей")

print("\n📊 ЛОГ 2 (Реакция: Al2O3 + CaF2 → AlF3 + CaO):")
print(f"   Уравнение из LLM: {llm_data_log2.get('balanced_equation')}")
total_log2 = sum(len(df) for df in log_2.values())
print(f"   Всего записей: {total_log2}")
for formula, df in log_2.items():
    names = llm_data_log2.get('compound_names', {}).get(formula, [])
    first_name = names[0] if names else "N/A"
    print(f"   - {formula} ({first_name}): {len(df)} записей")

print("\n" + "=" * 70)
print(f"✓ Общее количество загруженных записей: {total_log1 + total_log2}")
print("✓ Данные доступны в переменных: log_1 и log_2")
print("✓ Метаданные из LLM: llm_data_log1 и llm_data_log2")
print("=" * 70)

ИТОГОВАЯ СВОДКА ПО ЗАГРУЖЕННЫМ ДАННЫМ

📊 ЛОГ 1 (Реакция: MgO + SO2 → MgSO3):
   Уравнение из LLM: MgO + SO2 → MgSO3
   Всего записей: 1093
   - MgO (Magnesium oxide): 71 записей
   - SO2 (Sulfur dioxide): 1010 записей
   - MgSO3 (Magnesium sulfite): 12 записей

📊 ЛОГ 2 (Реакция: Al2O3 + CaF2 → AlF3 + CaO):
   Уравнение из LLM: Al2O3 + 3CaF2 → 2AlF3 + 3CaO
   Всего записей: 294
   - Al2O3 (Aluminium oxide): 100 записей
   - CaF2 (Calcium fluoride): 79 записей
   - AlF3 (Aluminium fluoride): 30 записей
   - CaO (Calcium oxide): 85 записей

✓ Общее количество загруженных записей: 1387
✓ Данные доступны в переменных: log_1 и log_2
✓ Метаданные из LLM: llm_data_log1 и llm_data_log2


## 5. Вспомогательные функции для термодинамических расчетов

In [20]:
def get_most_common_melting_boiling_points(df: pd.DataFrame) -> tuple:
    """
    Возвращает самое частое ненулевое значение MeltingPoint и BoilingPoint из DataFrame.
    Фильтрует некорректные значения: boiling_point должен быть больше melting_point.
    
    Args:
        df: DataFrame с данными вещества
        
    Returns:
        tuple: (melting_point, boiling_point) или (None, None)
    """
    melting_point = df[df['MeltingPoint'] != 0]['MeltingPoint'].value_counts().idxmax() if (df['MeltingPoint'] != 0).any() else None
    
    # Для boiling_point фильтруем только те значения, которые больше melting_point
    if melting_point is not None:
        boiling_candidates = df[(df['BoilingPoint'] != 0) & (df['BoilingPoint'] > melting_point)]['BoilingPoint']
    else:
        boiling_candidates = df[df['BoilingPoint'] != 0]['BoilingPoint']
    
    boiling_point = boiling_candidates.value_counts().idxmax() if len(boiling_candidates) > 0 else None
    
    return melting_point, boiling_point

print("✓ Функция get_most_common_melting_boiling_points определена")

✓ Функция get_most_common_melting_boiling_points определена


In [21]:
def calculate_thermodynamic_properties(row, T):
    """
    Расчет термодинамических свойств вещества при заданной температуре.
    
    Args:
        row: Строка DataFrame с коэффициентами и базовыми свойствами
        T: Температура в K
        
    Returns:
        tuple: (cp, enthalpy, entropy, gibbs_energy)
    """
    T = float(T)  # Ensure T is a float to allow negative powers
    T_ref = 298.15  # Reference temperature in K

    if 'Tmin' in row and 'Tmax' in row:
        if T < row['Tmin'] or T > row['Tmax']:
            print(f"Предупреждение: Температура {T} K выходит за пределы допустимого диапазона {row['Tmin']}-{row['Tmax']} K для вещества {row.get('Formula', '')}")

    # Coefficients for heat capacity calculation
    f1 = row['f1'] if 'f1' in row else 0
    f2 = row['f2'] if 'f2' in row else 0
    f3 = row['f3'] if 'f3' in row else 0
    f4 = row['f4'] if 'f4' in row else 0
    f5 = row['f5'] if 'f5' in row else 0
    f6 = row['f6'] if 'f6' in row else 0
    H298 = row['H298'] if 'H298' in row else 0
    S298 = row['S298'] if 'S298' in row else 0

    # Function to calculate Cp at any temperature
    def cp_function(temp):
        return (f1 + f2*temp/1000 + f3*temp**(-2) * 100_000 + 
                f4*temp**2 / 1_000_000 + f5*temp**(-3) * 1_000 + 
                f6*temp**3 * 10**(-9))

    # Current temperature heat capacity
    cp = cp_function(T)

    # If T equals reference temperature, no need for integration
    if abs(T - T_ref) < 1e-6:
        return cp, H298 * 1000, S298, H298 * 1000 - T*S298

    # Numerical integration for enthalpy change (ΔH)
    # ΔH = ∫(T_ref to T) Cp(T) dT
    num_points = 100  # Number of points for numerical integration
    temp_points = np.linspace(T_ref, T, num_points)
    cp_values = np.array([cp_function(t) for t in temp_points])
    delta_H = np.trapz(cp_values, temp_points)

    # Numerical integration for entropy change (ΔS)
    # ΔS = ∫(T_ref to T) Cp(T)/T dT
    cp_over_T = cp_values / temp_points
    delta_S = np.trapz(cp_over_T, temp_points)

    # Calculate final enthalpy and entropy
    enthalpy = H298 * 1000 + delta_H  # Convert H298 from kJ to J
    entropy = S298 + delta_S

    # Calculate Gibbs free energy
    gibbs_energy = enthalpy - T * entropy

    return cp, enthalpy, entropy, gibbs_energy

print("✓ Функция calculate_thermodynamic_properties определена")

✓ Функция calculate_thermodynamic_properties определена


In [22]:
def get_compound_records_for_range(df, t_range, melting, boiling, tolerance=1):
    """
    Возвращает список записей вещества, покрывающих весь температурный диапазон t_range.
    
    При достижении температуры фазового перехода (плавление/кипение) автоматически
    переключается на соответствующую фазу и продолжает поиск с температуры перехода.
    
    Args:
        df: DataFrame с данными вещества из БД
        t_range: Список [T_start, T_end] - начальная и конечная температура диапазона
        melting: Температура плавления (MeltingPoint) или None
        boiling: Температура кипения (BoilingPoint) или None
        tolerance: Допустимое отклонение при поиске записи по Tmin (по умолчанию 1K)
        
    Returns:
        List[pd.Series]: Список записей (строк DataFrame), покрывающих весь диапазон
    """
    records = []
    current_T = t_range[0]
    target_T = t_range[1]
    
    while current_T < target_T:
        # Определяем текущую фазу на основе температуры
        if boiling and current_T >= boiling:
            current_phase = 'g'
        elif melting and current_T >= melting:
            current_phase = 'l'
        else:
            current_phase = 's'
        
        # Ищем запись для текущей фазы, начинающуюся с current_T
        matching_records = df[
            (df['Phase'] == current_phase) & 
            (df['Tmin'] <= current_T + tolerance) & 
            (df['Tmin'] >= current_T - tolerance)
        ]
        
        if matching_records.empty:
            print(f"⚠ Не найдена запись для фазы '{current_phase}' при T={current_T} K")
            break
        
        # Берём первую подходящую запись
        record = matching_records.iloc[0]
        records.append(record)
        
        # Определяем следующую температуру
        record_tmax = record['Tmax']
        
        # Проверяем, достигли ли мы точки фазового перехода
        next_transition = None
        if melting and current_T < melting <= record_tmax:
            next_transition = melting
        elif boiling and current_T < boiling <= record_tmax:
            next_transition = boiling
        
        if next_transition:
            # Переход на следующую фазу с температуры перехода
            current_T = next_transition
            print(f"  Фазовый переход при T={next_transition} K: {current_phase} → ", end="")
            print('l' if next_transition == melting else 'g')
        else:
            # Переход к следующей записи той же фазы
            current_T = record_tmax
            
            # Если достигли или превысили целевую температуру, завершаем
            if current_T >= target_T:
                break
    
    return records

print("✓ Функция get_compound_records_for_range определена")

✓ Функция get_compound_records_for_range определена


## 6. Пример использования функций

Тестируем функции на данных AlF3 из Лога 2.

In [23]:
# Получаем данные AlF3 из Лога 2
AlF3 = log_2.get('AlF3', pd.DataFrame())

if AlF3.empty:
    print("⚠ Данные AlF3 не найдены")
else:
    # Определяем точки фазовых переходов
    melting, boiling = get_most_common_melting_boiling_points(AlF3)
    
    print("=" * 70)
    print("АНАЛИЗ AlF3")
    print("=" * 70)
    print(f"Точки фазовых переходов:")
    print(f"  Плавление: {melting} K" if melting else "  Плавление: не определено")
    print(f"  Кипение: {boiling} K" if boiling else "  Кипение: не определено")
    print(f"\nВсего записей в БД: {len(AlF3)}")
    print(f"Фазы: {list(AlF3['Phase'].unique())}")

АНАЛИЗ AlF3
Точки фазовых переходов:
  Плавление: 2523.0 K
  Кипение: не определено

Всего записей в БД: 30
Фазы: ['s', 'l']


In [24]:
# Тест функции get_compound_records_for_range
t_range = [298, 2500]

print("=" * 70)
print(f"ТЕСТ: Получение записей AlF3 для диапазона {t_range} K")
print("=" * 70)

if not AlF3.empty:
    compound_records = get_compound_records_for_range(AlF3, t_range, melting, boiling)
    
    print(f"\n✓ Найдено {len(compound_records)} записей:")
    for i, record in enumerate(compound_records, 1):
        print(f"  {i}. Фаза: {record['Phase']}, Tmin: {record['Tmin']} K, Tmax: {record['Tmax']} K")
else:
    print("⚠ Невозможно выполнить тест - данные AlF3 отсутствуют")

ТЕСТ: Получение записей AlF3 для диапазона [298, 2500] K
  Фазовый переход при T=2523.0 K: s → l

✓ Найдено 3 записей:
  1. Фаза: s, Tmin: 298.15 K, Tmax: 500.0 K
  2. Фаза: s, Tmin: 500.0 K, Tmax: 728.0 K
  3. Фаза: s, Tmin: 728.0 K, Tmax: 2523.0 K
